In [39]:
import os
import json
import numpy as np
import pickle 
import tqdm
import gzip
import tarfile

In [22]:
# with tarfile.open("../data/parsed/opendota_parsed_matches.tar.gz", 'r:gz') as zfp:
#     for filename in zfp.getnames():
#         match = pickle.load(zfp.extractfile(filename))
#         print(match)
#         break

zfp = tarfile.open("../data/parsed/opendota_parsed_matches.tar.gz", 'r:gz')
all_filenames = zfp.getnames()

In [23]:
filenames = all_filenames[1:10]

In [24]:
filenames

['parsed/41990.pkl',
 'parsed/32501.pkl',
 'parsed/14503.pkl',
 'parsed/26409.pkl',
 'parsed/23168.pkl',
 'parsed/22720.pkl',
 'parsed/15572.pkl',
 'parsed/45219.pkl',
 'parsed/28569.pkl']

In [25]:
fp = zfp.extractfile(filenames[0])

In [41]:
fp.seek(0)
match = pickle.loads(fp.read())
match


{'match_id': 6235518304,
 'barracks_status_dire': 63,
 'barracks_status_radiant': 3,
 'chat': [{'time': -62,
   'type': 'chat',
   'key': 'pa',
   'slot': 5,
   'player_slot': 128},
  {'time': -59, 'type': 'chat', 'key': '<3', 'slot': 5, 'player_slot': 128},
  {'time': -36, 'type': 'chat', 'key': 'Ya', 'slot': 1, 'player_slot': 1},
  {'time': -27,
   'type': 'chat',
   'key': 'i love you',
   'slot': 5,
   'player_slot': 128},
  {'time': -19,
   'type': 'chat',
   'key': 'we can marry?',
   'slot': 5,
   'player_slot': 128},
  {'time': -15, 'type': 'chat', 'key': 'ya', 'slot': 1, 'player_slot': 1},
  {'time': -13,
   'type': 'chat',
   'key': 'come pv',
   'slot': 5,
   'player_slot': 128},
  {'time': -12, 'type': 'chat', 'key': 'nohomo', 'slot': 1, 'player_slot': 1},
  {'time': -2,
   'type': 'chat',
   'key': 'i can suck ur dick',
   'slot': 5,
   'player_slot': 128},
  {'time': -1,
   'type': 'chat',
   'key': 'dont worry',
   'slot': 5,
   'player_slot': 128},
  {'time': 23,
   'ty

In [76]:
def get_rank(rank: int):
    oh = np.zeros(5)
    oh[rank] = 1
    return oh

def extract_data_point(match):
    players = match["players"]
    t0 = list(filter(lambda p: p["isRadiant"], players))
    t1 = list(filter(lambda p: not p["isRadiant"], players))
    team = [t0, t1]

    xs = []
    ys = []
    for i in [0, 1]:
        hids = [p["hero_id"] for p in team[i]]
        gpm_rank = sorted([(p["hero_id"], p["gold_per_min"]) for p in team[i]], key=lambda x: x[1], reverse=False)
        xpm_rank = sorted([(p["hero_id"], p["xp_per_min"]) for p in team[i]], key=lambda x: x[1], reverse=False)
        ka_rank = sorted([(p["hero_id"], p["kills"] + p["assists"]) for p in team[i]], key=lambda x: x[1], reverse=False)
        d_rank = sorted([(p["hero_id"], p["deaths"]) for p in team[i]], key=lambda x: x[1], reverse=False)
        lasthits_rank = sorted([(p["hero_id"], p["last_hits"]) for p in team[i]], key=lambda x: x[1], reverse=False)
        wards_placed = []
        for p in team[i]:
            item_uses = p["item_uses"]
            c = 0
            if "ward_observer" in item_uses:
                c += item_uses["ward_observer"]
            if "ward_sentry" in item_uses:
                c += item_uses["ward_sentry"]
            wards_placed.append((p["hero_id"], c))
        
        for h in team[i]:
            features = []
            hid = h["hero_id"]

            onehot_heroes = np.zeros((136))
            for team_hid in hids:
                onehot_heroes[team_hid] = 1
            features.append(onehot_heroes)
            features.append(get_rank(gpm_rank.index((hid, h["gold_per_min"]))))
            features.append(get_rank(xpm_rank.index((hid, h["xp_per_min"]))))
            features.append(get_rank(ka_rank.index((hid,  h["kills"] + h["assists"]))))
            features.append(get_rank(d_rank.index((hid, h["deaths"]))))
            features.append(get_rank(lasthits_rank.index((hid, h["last_hits"]))))

            if hid == wards_placed[0][0]:
                y = 5
            else:
                y = h["lane_role"]
            xs.append(np.concatenate(features))
            ys.append(y)

    return xs, ys


x_train, y_train = extract_data_point(match)
for r in x_train:
    print(r.shape)

x_stack = np.vstack(x_train)

(161,)
(161,)
(161,)
(161,)
(161,)
(161,)
(161,)
(161,)
(161,)
(161,)


[5, 1, 3, 3, 1, 5, 1, 3, 3, 2]

In [78]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score,cross_val_predict, train_test_split
from sklearn.metrics import confusion_matrix


clf = DecisionTreeClassifier(random_state=0)
cross_val_score(clf, x_stack, y_train)

ValueError: n_splits=5 cannot be greater than the number of members in each class.